In [40]:
from numpy import ndarray
from math import cos, sin, atan2
from typing import TypeAlias

class DimensionError(Exception):
	pass

class Vector:
	def __init__(self, *args: int | float):
		if len(args) == 1 and isinstance(args[0], Vector):
			self.__args = args[0].__args
		if any(not isinstance(arg, int | float) for arg in args):
			raise TypeError(f"Vector.__init__: {args}")
		
		self.__args = args
	
	@staticmethod
	def polar(r, theta, phi=None) -> "Vector":
		if phi is None:
			return Vector(r*cos(theta), r*sin(theta))
		else:
			return Vector(r*sin(theta)*cos(phi), r**sin(theta)*sin(phi), r*cos(theta))
	
	def _repr_latex_(self) -> str:
		_content = '\\\\'.join(map(str, self.__args))
		return f"$\\begin{{bmatrix}} {_content} \\end{{bmatrix}}$"
	
	@property
	def dim(self) -> int:
		return len(self.__args)
	
	@property
	def e(self) -> tuple:
		return self.__args
	
	@property
	def i(self) -> float:
		return self.__args[0]
	
	@property
	def j(self) -> float:
		return self.__args[1]
	
	@property
	def k(self) -> float:
		return self.__args[2]
	
	@property
	def r(self) -> float:
		return sum(arg ** 2 for arg in self.__args) ** 0.5
	
	@property
	def theta(self) -> float:
		if self.dim == 2:
			return atan2(self.j, self.i)
		elif self.dim == 3:
			return atan2((self.i**2 + self.j**2) ** 0.5, self.k)
		else:
			raise DimensionError(f"input: {len(self)}, expected: 2")
	
	@property
	def phi(self) -> float:
		if self.dim == 3:
			return atan2(self.j, self.i)
		else:
			raise DimensionError(f"input: {len(self)}, expected: 3")
	
	def __abs__(self) -> float:
		return self.r
	
	def __len__(self) -> int:
		return self.dim
	
	def __pos__(self) -> "Vector":
		return self
	
	def __neg__(self) -> "Vector":
		return Vector(*[-arg for arg in self.__args])
	
	def __add__(self, other) -> "Vector":
		if isinstance(other, Vector):
			if len(self) != len(other):
				raise ValueError(f"Vector.__add__: {len(self)} != {len(other)}")
			
			return Vector(*[a + b for a, b in zip(self.__args, other.__args)])
		elif isinstance(other, Iterable):
			if len(self) != len(other):
				raise ValueError(f"Vector.__add__: {len(self)} != {len(other)}")
			
			return Vector(*[a + b for a, b in zip(self.__args, other)])
		else:
			return NotImplemented
	
	def __radd__(self, other) -> "Vector":
		return self.__add__(other)
	
	def __sub__(self, other) -> "Vector":
		return self.__add__(-other)
	
	def __rsub__(self, other) -> "Vector":
		return -(self.__add__(-other))
	
	def __matmul__(self, other) -> float:
		if isinstance(other, Vector):
			if len(self) != len(other):
				raise ValueError(f"Vector.__matmul__: {len(self)} != {len(other)}")
			
			return sum(a * b for a, b in zip(self.__args, other.__args))
		else:
			return NotImplemented
	
	def __rmatmul__(self, other) -> float:
		return self.__matmul__(other)
	
	def __mul__(self, other) -> "Vector":
		if isinstance(other, Vector):
			if self.dim != other.dim:
				raise ValueError(f"Vector.__mul__: {self.dim} != {other.dim}")
			
			if self.dim == 3:
				return Vector(
					other.k * self.j - other.j * self.k,
					other.i * self.k - other.k * self.i,
					other.j * self.i - other.i * self.j
				)
			else:
				raise DimensionError(f"Vector.__mul__: {self.dim}")
		else:
			return Vector(*[a * other for a in self.__args])
		
	def __rmul__(self, other) -> "Vector":
		return -self.__mul__(other)
	
	def __truediv__(self, other) -> "Vector":
		if isinstance(other, Iterable | VecLike):
			raise TypeError(f"Vector.__truediv__: {type(other)}")
			
		return self.__mul__(1 / other)
	
	def __repr__(self) -> str:
		return f"{self}"
	
	def __str__(self) -> str:
		return self.__repr__()
	
	def __format__(self, format_spec) -> str:
		if format_spec.endswith('p'):  # polar coordinates
			if self.dim not in (2, 3):
				raise DimensionError(f"Vector.__format__: {self.dim}")
			
			options = '{:' + format_spec[:-1] + '}'
			text = [options.format(self.r), options.format(self.theta)]
			
			if self.dim == 3:
				text.append(options.format(self.phi))
				return f"({text[0]}, {text[1]}, {text[2]})"
			else:
				return f"({text[0]}, {text[1]})"
		elif format_spec.endswith('u'):  # unit vector
			options = '{:' + format_spec[:-1] + '}'
			text = [options.format(_e) for _e in self.__args]
			
			subscripts = '₀₁₂₃₄₅₆₇₈₉'
			ret = ' + '.join(f"{txt} e{''.join(subscripts[int(c)] for c in str(i))}" for i, txt in enumerate(text))
			
			return '(' + ret + ')'
		else:
			ret = f"{', '.join(map(lambda x: ('{:' + format_spec + '}').format(x), self.__args))}"
			
			return '(' + ret + ')'
	
	def __iter__(self) -> iter:
		return iter(self.__args)

	def __reversed__(self) -> "Vector":
		return Vector(*reversed(self.__args))

	def __eq__(self, other) -> bool:
		if not isinstance(other, Vector):
			return False
		else:
			return self.__args == other.__args

	def __lt__(self, other) -> bool:
		if not isinstance(other, Vector):
			raise TypeError
		
		if self.dim != other.dim:
			raise DimensionError
		
		return abs(self) < abs(other)

	def __le__(self, other):
		return self == other or self < other

	def __gt__(self, other):
		return not (self <= other)

	def __ge__(self, other):
		return not (self < other)

	def __int__(self) -> int:
		return int(abs(self))
	
	def __float__(self) -> float:
		return float(abs(self))
	
	def __bool__(self):
		return bool(self.r)
	
	def __complex__(self) -> complex:
		if self.dim == 2:
			return complex(self.i, self.j)
		else:
			raise DimensionError(f"input: {len(self)}, expected: 2")

Iterable: TypeAlias = tuple | list | ndarray
VecLike: TypeAlias = Vector | tuple

from math import pi

class UnitError(Exception):
	pass

class Unit:
	def __init__(self, *, kg=0., m=0., s=0., A=0., K=0., mol=0., cd=0., rad=0.):
		self.__data: dict[str, float] = {
			'kg': kg,
			'm': m,
			's': s,
			'A': A,
			'K': K,
			'mol': mol,
			'cd': cd,
			'rad': rad
		}

	@property
	def kg(self) -> float:
		return self.__data['kg']

	@property
	def m(self) -> float:
		return self.__data['m']
	
	@property
	def s(self) -> float:
		return self.__data['s']
	
	@property
	def A(self) -> float:
		return self.__data['A']
	
	@property
	def K(self) -> float:
		return self.__data['K']
	
	@property
	def mol(self) -> float:
		return self.__data['mol']
	
	@property
	def cd(self) -> float:
		return self.__data['cd']
	
	@property
	def rad(self) -> float:
		return self.__data['rad']
	
	def __bool__(self) -> bool:
		return any(self.__data.values())

	def __repr__(self) -> str:
		_mul = '*'  # '⋅'
		_pow = '**'  # '^'
		
		ret = ["", ""]
		
		for name, power in self.__data.items():
			if power == 0:
				continue
			elif abs(power) == 1:
				ret[power < 0] += f"{name}{_mul}"
			else:
				ret[power < 0] += f'{name}{_pow}{int(abs(power)) if power.is_integer() else abs(power)}{_mul}'
		
		front, back = ret
		front = front.strip(_mul)
		back = back.strip(_mul)
		
		return f"{front}{'/' if back else ''}{back}".strip()

	def __str__(self) -> str:
		return self.__repr__()

	def __iter__(self) -> iter:
		yield from self.__data.items()

	def __pos__(self) -> "Unit":
		return self
	
	def __neg__(self) -> "Unit":
		return Unit(**{name: -power for name, power in self})

	def __eq__(self, other) -> bool:
		if isinstance(other, Unit):
			return self.__data == other.__data
		else:
			return NotImplemented

	def __mul__(self, other) -> "Unit":
		if isinstance(other, Unit):
			return Unit(**{name: power + other.__data[name] for name, power in self})
		else:
			return NotImplemented

	def __rmul__(self, other) -> "Quantity | Unit":
		if isinstance(other, int | float):
			return Quantity(other, self)
		elif isinstance(other, VecLike):
			return Quantity(Vector(*other), self)
		else:
			return self.__mul__(other)

	def __truediv__(self, other) -> "Unit":
		return self.__mul__(-other)

	def __rtruediv__(self, other) -> "Quantity | Unit":
		if isinstance(other, int | float):
			return Quantity(other, -self)
		elif isinstance(other, VecLike):
			return Quantity(Vector(*other), -self)
		else:
			return NotImplemented

	def __pow__(self, other) -> "Unit":
		if isinstance(other, int | float):
			return Unit(**{name: power * other for name, power in self})
		else:
			return NotImplemented

class Quantity:
	presets: dict[str, Unit] = {}
	
	def __init__(self, value: float | Vector, unit: Unit):
		self.__value = value
		self.__unit = unit

	def is_vector(self, unit: Unit = None) -> bool:
		return isinstance(self.__value, Vector) and (unit is None or self.unit == unit)
	
	def is_scalar(self, unit: Unit = None) -> bool:
		return not self.is_vector() and (unit is None or self.unit == unit)

	@property
	def value(self) -> float | Vector:
		return self.__value
	
	@property
	def unit(self) -> Unit:
		return self.__unit
	
	@property
	def e(self):
		return tuple(v * self.unit for v in self.__value)
	
	def __iter__(self):
		yield self.e
	
	def __bool__(self):
		return bool(self.__value)
	
	def __len__(self):
		return len(self.__value)
	
	def __abs__(self):
		return Quantity(abs(self.__value), self.__unit)

	def _repr_latex_(self):
		_content = str(self.unit).replace('**', '^').replace('*', ' \\cdot ')
		return f"${self.value}~~{_content}$"

	def __format__(self, format_spec):
		return f"{self.__value:{format_spec}} [{self.__unit}]"
	
	def __repr__(self) -> str:
		return self.__format__('')
	
	def __str__(self) -> str:
		return self.__repr__()
	
	def __pos__(self) -> "Quantity":
		return self
	
	def __neg__(self) -> "Quantity":
		return Quantity(-self.__value, self.__unit)
	
	def __eq__(self, other) -> bool:
		if isinstance(other, Quantity):
			if self.__value == other.__value:
				return True if self.__value == 0 else self.__unit == other.__unit
		elif isinstance(other, int | float):
			return self.__value == 0 and other == 0
		else:
			return NotImplemented
		
	def __lt__(self, other) -> bool:
		if isinstance(other, Quantity):
			if self.__unit == other.__unit:
				return self.__value < other.__value
			else:
				raise UnitError(f"Cannot compare {self.__unit} and {other.__unit}.")
		else:
			return self.__value < other
		
	def __gt__(self, other) -> bool:
		return other < self
		
	def __le__(self, other) -> bool:
		return self == other or self < other
	
	def __ge__(self, other) -> bool:
		return self == other or self > other
	
	def __add__(self, other) -> "Quantity":
		if isinstance(other, Quantity):
			if self.__unit == other.__unit:
				return Quantity(self.__value + other.__value, self.__unit)
			else:
				raise UnitError(f"Cannot add {self.__unit} and {other.__unit}.")
		else:
			return Quantity(self.__value + other, self.__unit)

	def __radd__(self, other) -> "Quantity":
		return self.__add__(other)

	def __sub__(self, other) -> "Quantity":
		return self.__add__(-other)

	def __rsub__(self, other) -> "Quantity":
		return -(self.__add__(-other))
	
	def __mul__(self, other) -> "Quantity":
		if isinstance(other, Quantity):
			return Quantity(self.__value * other.__value, self.__unit * other.__unit)
		elif isinstance(other, VecLike):
			return Quantity(self.__value * Vector(*other), self.__unit)
		elif isinstance(other, int | float):
			return Quantity(self.__value * other, self.__unit)
		elif isinstance(other, Iterable):
			for i in range(len(other)):
				other[i] *= self
			return other
		else:
			return NotImplemented

	def __rmul__(self, other) -> "Quantity":
		return self.__mul__(other)

	def __truediv__(self, other) -> "Quantity":
		return self.__mul__(1 / other)
	
	def __rtruediv__(self, other) -> "Quantity":
		if isinstance(other, VecLike):
			return Quantity(Vector(*other) / self.__value, -self.__unit)
		elif isinstance(other, int | float):
			return Quantity(other / self.__value, -self.__unit)
		else:
			return Quantity(other / self.__value, -self.__unit)

	def __pow__(self, other) -> "Quantity":
		if isinstance(other, int | float):
			return Quantity(self.__value ** other, self.__unit ** other)
		else:
			return NotImplemented

prefixes = {
	'Y': 1e24,
	'Z': 1e21,
	'E': 1e18,
	'P': 1e15,
	'T': 1e12,
	'G': 1e9,
	'M': 1e6,
	'k': 1e3,
	'h': 1e2,
	'd': 1e-1,
	'c': 1e-2,
	'm': 1e-3,
	'µ': 1e-6,
	'n': 1e-9,
	'p': 1e-12,
	'f': 1e-15,
	'a': 1e-18,
	'z': 1e-21,
	'y': 1e-24
}

kg = Unit(kg=1)
m = Unit(m=1)
s = Unit(s=1)
A = Unit(A=1)
K = Unit(K=1)
mol = Unit(mol=1)
cd = Unit(cd=1)
rad = Unit(rad=1)

Quantity.presets['N'] = kg * m/s**2
Quantity.presets['J'] = (kg * m/s**2) * m
Quantity.presets['Pa'] = (kg * m/s**2) / m**2

In [42]:
N = kg*m/s**2

(3, 5)*N

(3, 5) [kg*m/s**2]

$\mathrm{3.5~~kg\cdot m/s^2}$

In [27]:
Vector(1.3, 2.3, 4.5)

(1.3, 2.3, 4.5)

In [45]:
display(3)

3